In [2]:
import os
import pandas as pd

df_sample = pd.read_table("Z:/Documents/GitHub/toy/data/distance-data.tsv")
class_pair = list()
i = 0
for class1 in range(104):
  for class2 in range(class1):
    class_pair.append([class1, class2, df_sample.scaledPatristicDistance[i]])
    i=i+1
import pandas as pd

In [12]:

def get_key_for_class2distance(TrueClass, PredClass):
    TrueClass = tf.cast(TrueClass, tf.string)
    PredClass = tf.cast(PredClass, tf.string)
    out = tf.strings.reduce_join(
				axis = -1,
				inputs = tf.stack([TrueClass, PredClass], axis = 1),
				separator = ' <=> '
			  )
    return out

In [7]:
import tensorflow as tf

In [5]:
df_test = pd.DataFrame(class_pair).rename(
    columns = {0:'class1', 1:'class2', 2:'scaledPatristicDistance' })

In [ ]:
import toy

c1 = tf.convert_to_tensor(df_test.class1)
c2 = tf.convert_to_tensor(df_test.class2)
get_key_for_class2distance(c1,c2)


In [11]:
tf.convert_to_tensor(df_test.class1)

<tf.Tensor: shape=(5356,), dtype=int64, numpy=array([  1,   2,   2, ..., 103, 103, 103], dtype=int64)>

In [30]:
c1 = tf.constant(df_test.class1, dtype = tf.int32)
c2 = tf.constant(df_test.class2, dtype = tf.int32)
v1 = tf.constant(df_test.scaledPatristicDistance, dtype = tf.float32)

type(c1)

c1 = tf.strings.as_string(df_test.class1)
c2 = tf.strings.as_string(df_test.class2)
toy.get_key_for_class2distance(c1, c2)

<tf.Tensor: shape=(5356,), dtype=string, numpy=
array([b'1 <=> 0', b'2 <=> 0', b'2 <=> 1', ..., b'103 <=> 100',
       b'103 <=> 101', b'103 <=> 102'], dtype=object)>

In [55]:
def get_key_for_class2distance(TrueClass, PredClass):

    TrueClass = tf.strings.as_string(TrueClass)
    PredClass = tf.strings.as_string(PredClass)
    out = tf.strings.reduce_join(
				axis = -1,
				inputs = tf.stack([TrueClass, PredClass], axis = 1),
				separator = ' <=> '
			  )
    return out

def Get_Distance(TrueClass, PredClass, LOOKUPTB, BATCH_SIZE):
    TrueClass = tf.strings.as_string(TrueClass)
    PredClass = tf.strings.as_string(PredClass)
    return tf.reshape(tf.math.maximum(
			LOOKUPTB.lookup(tf.strings.reduce_join(
				axis = -1,
				inputs = tf.stack([TrueClass, PredClass], axis = 1),
				separator = ' <=> '
			)),
			LOOKUPTB.lookup(tf.strings.reduce_join(
				axis = -1,
				inputs = tf.stack([PredClass, TrueClass], axis = 1),
				separator = ' <=> '
			))
            ),(BATCH_SIZE, 1))


lookup = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
    get_key_for_class2distance(df_test.class1, df_test.class2),
    tf.constant(df_test.scaledPatristicDistance)
    ),
    default_value = -1
)

lookup.export()

batch_size = 64
import random
pred = random.sample(range(104), k = batch_size)
real = random.sample(range(104), k = batch_size)
pred_onehot = tf.one_hot(pred, 104)
real_onehot = tf.one_hot(real, 104)
distance = Get_Distance(real, pred , LOOKUPTB=lookup,
            BATCH_SIZE = batch_size)

ce_loss = tf.keras.losses.CategoricalCrossentropy()

ce_loss(real_onehot, pred_onehot), ce_loss(real_onehot, pred_onehot, sample_weight = distance)
#toy.pd_categorical_crossentropy(pred, real)


(<tf.Tensor: shape=(), dtype=float32, numpy=16.118095>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.655802>)